# Hashing Vectorizer example
In this notebook, we'll experiment with HashingVectorizer by making a classifier that predicts whether a text chunk comes from the English Wikipedia articles ["Anarchism"](https://en.wikipedia.org/wiki/Anarchism) or ["Anachronism"](https://en.wikipedia.org/wiki/Anachronism).

## Preamble
First, let's import everything that we'll need.

In [2]:
import random

import mwapi
import mwparserfromhell as mwparser
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import HashingVectorizer

ImportError: No module named 'mwapi'

## Get the text from the API

In [2]:
session = mwapi.Session("https://en.wikipedia.org", 
                        user_agent="Hashing vectorizer example <aaron.halfaker@gmail.com>")

doc = session.get(action="query", prop="revisions", titles=["Anarchism", "Anachronism"], rvprop=['content'])

anarchism_text = doc['query']['pages']['12']['revisions'][0]['*']
anarchonism_text = doc['query']['pages']['60731']['revisions'][0]['*']

## Build up a set of observations

In [3]:
observations = []
for text_chunk in mwparser.parse(anarchism_text).filter_text():
    text = text_chunk.value
    if len(text) > 25:
        observations.append((text, "anarchism"))
for text_chunk in mwparser.parse(anarchonism_text).filter_text():
    text = text_chunk.value
    if len(text) > 25:
        observations.append((text, "anachronism"))

print("anarchism paragraphs:", sum(1 for _, label in observations if label == "anarchism"))
print("anachronism paragraphs:", sum(1 for _, label in observations if label == "anachronism"))

anarchism paragraphs: 1433
anachronism paragraphs: 119


## Split into train/test

In [4]:
random.shuffle(observations)
train_set = observations[:int(len(observations)*0.8)]
test_set = observations[int(len(observations)*0.8):]
len(train_set), len(test_set)

(1241, 311)

## HashingVectorizer and prediction model
We'll use a GradientBoosting model because it ought to work pretty well

In [5]:
hv = HashingVectorizer(n_features=2*16)
gbc = GradientBoostingClassifier()

## Train the classifier
We'll set the "sample weight" to be proportionally stronger for "Anachronism" because we don't have as many observations.

In [6]:
# Training
texts, labels_y = zip(*train_set)
features_X = hv.transform(texts)
gbc.fit(features_X, labels_y, 
        sample_weight=[119/(119+1433) if l == "anarchism" else 1433/(119+1433) for l in labels_y])

GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

## Test the classifier
This score() method generates a simple accuracy measure.  This result suggest that we can predict ~86% of the test set correctly.

In [7]:
# Testing
texts, labels_y = zip(*test_set)
features_X = hv.transform(texts)
gbc.score(features_X.todense(), labels_y)

0.86173633440514474

Statistics are great, but let's look at some example predictions.  This loop generates predictions 

In [15]:
for text, label in test_set[:10]:
    features_X = hv.transform([text])
    print("text:", repr(text[:50] + "..."), "\n",
          "\tactual:", label, "\n",
          "\tprediction:", dict(zip(gbc.classes_, 
                                    [int(v*100) for v in gbc.predict_proba(features_X.todense())[0]])))
    

text: 'University of Toronto Press ...' 
 	actual: anarchism 
 	prediction: {'anachronism': 32, 'anarchism': 67}
text: 'http://robertgraham.wordpress.com/anarchism-a-docu...' 
 	actual: anarchism 
 	prediction: {'anachronism': 18, 'anarchism': 81}
text: "Robert Graham's Anarchism Weblog..." 
 	actual: anarchism 
 	prediction: {'anachronism': 32, 'anarchism': 67}
text: ' without regard for God, state, or morality....' 
 	actual: anarchism 
 	prediction: {'anachronism': 56, 'anarchism': 43}
text: 'Enquiry Concerning Political Justice and its Influ...' 
 	actual: anarchism 
 	prediction: {'anachronism': 33, 'anarchism': 66}
text: 'Anarchism: A Documentary History of Libertarian Id...' 
 	actual: anarchism 
 	prediction: {'anachronism': 19, 'anarchism': 80}
text: 'The Escuela Moderna Movement of Francisco Ferrer: ...' 
 	actual: anarchism 
 	prediction: {'anachronism': 26, 'anarchism': 73}
text: '"Anarchism, insurrections and insurrectionalism" b...' 
 	actual: anarchism 
 	prediction: {'a